In [1]:
#Dependencies
import pandas as pd
import numpy as np

In [2]:
# Name of the csv file
crime_csv_path = 'dc-crimes-search-results.csv'

In [3]:
# Import the data into a Pandas DataFrame
crime_df = pd.read_csv(crime_csv_path)
crime_df.head()

,NEIGHBORHOOD_CLUSTER,CENSUS_TRACT,offensegroup,LONGITUDE,END_DATE,offense-text,SHIFT,YBLOCK,DISTRICT,WARD,...,XBLOCK,BLOCK,START_DATE,CCN,OFFENSE,ANC,REPORT_DAT,METHOD,location,LATITUDE
0,cluster 15,600.0,property,-77.069766,2015-05-05T17:45:00.000,theft f/auto,evening,140519.000000,2.0,3.0,...,393951.000000,3500 - 3599 block of lowell street nw,2015-05-05T15:00:00.000,15064669,theft f/auto,3C,2015-05-05T22:06:00.000Z,others,"38.932540365033717,-77.069768169731375",38.932533
1,cluster 2,3000.0,violent,-77.031208,2015-05-05T20:55:00.000,robbery,evening,140084.000000,3.0,1.0,...,397294.000000,1300 - 1399 block of irving street nw,2015-05-05T20:50:00.000,15064781,robbery,1A,2015-05-06T01:05:00.000Z,gun,"38.928638407177509,-77.031210110921407",38.928631
2,cluster 3,4400.0,property,-77.028460,2015-05-05T22:14:00.000,motor vehicle theft,evening,139201.000000,3.0,1.0,...,397532.000000,1200 - 1247 block of florida avenue nw,2015-05-05T18:18:00.000,15064796,motor vehicle theft,1B,2015-05-06T02:58:00.000Z,others,"38.920684762113531,-77.028462123394547",38.920677
3,cluster 23,8804.0,violent,-76.985496,2015-06-23T08:00:00.000,homicide,midnight,137689.000000,5.0,5.0,...,401258.000000,1200 - 1299 block of holbrook terrace ne,2015-06-23T05:23:00.000,15094190,homicide,5D,2015-06-24T04:00:00.000Z,gun,"38.907066722563066,-76.985498377563218",38.907059
4,cluster 25,8100.0,property,-76.990245,2015-06-23T06:30:00.000,theft f/auto,day,136398.650012,1.0,6.0,...,400846.210015,duncan place ne and 12th street ne,2015-06-22T22:00:00.000,15094194,theft f/auto,6A,2015-06-23T12:48:00.000Z,others,"38.895443280295112,-76.990247632314322",38.895435


In [4]:
# Delete extraneous column
crime_df.count()

NEIGHBORHOOD_CLUSTER    279042
CENSUS_TRACT            281769
offensegroup            282611
LONGITUDE               282611
END_DATE                271780
offense-text            282611
SHIFT                   282611
YBLOCK                  282611
DISTRICT                282449
WARD                    282599
YEAR                    282611
offensekey              282611
BID                      46291
sector                  282432
PSA                     282432
ucr-rank                282611
BLOCK_GROUP             281769
VOTING_PRECINCT         282551
XBLOCK                  282611
BLOCK                   282611
START_DATE              282603
CCN                     282611
OFFENSE                 282611
ANC                     282611
REPORT_DAT              282611
METHOD                  282611
location                282611
LATITUDE                282611
dtype: int64

In [5]:
# drop all rows with missing information
no_null_crime_df = crime_df.dropna(how='any')

In [6]:
#Verify dropped rows
no_null_crime_df.count()

NEIGHBORHOOD_CLUSTER    43496
CENSUS_TRACT            43496
offensegroup            43496
LONGITUDE               43496
END_DATE                43496
offense-text            43496
SHIFT                   43496
YBLOCK                  43496
DISTRICT                43496
WARD                    43496
YEAR                    43496
offensekey              43496
BID                     43496
sector                  43496
PSA                     43496
ucr-rank                43496
BLOCK_GROUP             43496
VOTING_PRECINCT         43496
XBLOCK                  43496
BLOCK                   43496
START_DATE              43496
CCN                     43496
OFFENSE                 43496
ANC                     43496
REPORT_DAT              43496
METHOD                  43496
location                43496
LATITUDE                43496
dtype: int64

In [7]:
# Verify the data types
no_null_crime_df.dtypes


NEIGHBORHOOD_CLUSTER     object
CENSUS_TRACT            float64
offensegroup             object
LONGITUDE               float64
END_DATE                 object
offense-text             object
SHIFT                    object
YBLOCK                  float64
DISTRICT                float64
WARD                    float64
YEAR                      int64
offensekey               object
BID                      object
sector                   object
PSA                     float64
ucr-rank                  int64
BLOCK_GROUP              object
VOTING_PRECINCT          object
XBLOCK                  float64
BLOCK                    object
START_DATE               object
CCN                       int64
OFFENSE                  object
ANC                      object
REPORT_DAT               object
METHOD                   object
location                 object
LATITUDE                float64
dtype: object

In [8]:
# Check to see if there are any values with mispelled or similar values in "Offense Type"
no_null_crime_df["OFFENSE"].value_counts()

theft/other                   25727
theft f/auto                  10021
robbery                        2778
motor vehicle theft            1679
assault w/dangerous weapon     1527
burglary                       1465
sex abuse                       242
homicide                         47
arson                            10
Name: OFFENSE, dtype: int64

In [9]:
#Clean up Neighborhood Cluster and replace it with actual name areas
no_null_crime_df=no_null_crime_df.rename(index=str , columns={"NEIGHBORHOOD_CLUSTER":"NEIGHBORHOOD"})



In [10]:
print(no_null_crime_df.head())
# replace the clusters with actual names continued
no_null_crime_df['NEIGHBORHOOD']= no_null_crime_df['NEIGHBORHOOD'].replace(
    {'cluster 15':'Cleveland Park, Woodley Park, Massachusetts Avenue Heights, Woodland-Normanstone Terrace',
     'cluster 2':'Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View',
     'cluster 3':'Howard University, Le Droit Park, Cardozo/Shaw',
     'cluster 4':'Georgetown, Burleith/Hillandale',
     'cluster 5':'West End, Foggy Bottom, GWU',
     'cluster 6':'Dupont Circle, Connecticut Avenue/K Street',
     'cluster 1':'Kalorama Heights, Adams Morgan, Lanier Heights',
     'cluster 7':'Shaw, Logan Circle',
     'cluster 8': 'Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street',
     'cluster 9': 'Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point',
     'cluster 10': 'Hawthorne, Barnaby Woods, Chevy Chase',
     'cluster 11':'Friendship Heights, American University Park, Tenleytown',
     'cluster 12':'North Cleveland Park, Forest Hills, Van Ness',
     'cluster 13':'Spring Valley, Palisades, Wesley Heights, Foxhall Crescent, Foxhall Village, Georgetown Reservoir',
     'cluster 14':'Cathedral Heights, McLean Gardens, Glover Park',
     'cluster 16': 'Colonial Village, Shepherd Park, North Portal Estates',
     'cluster 17': 'Takoma, Brightwood, Manor Park',
     'cluster 18': 'Brightwood Park, Crestwood, Petworth',
     'cluster 19':'Lamond Riggs, Queens Chapel, Fort Totten, Pleasant Hill',
     'cluster 20': 'North Michigan Park, Michigan Park, University Heights',
     'cluster 21': 'Edgewood, Bloomingdale, Truxton Circle, Eckington',
     'cluster 22':'Brookland, Brentwood, Langdon',
     'cluster 23':'Ivy City, Arboretum, Trinidad, Carver Langston',
     'cluster 24':'Woodridge, Fort Lincoln, Gateway',
     'cluster 25': 'NoMa, Union Station, Stanton Park, Kingman Park',
     'cluster 26':'Capitol Hill, Lincoln Park',
     'cluster 27':'Near Southeast, Navy Yard',
     'cluster 28': 'Historic Anacostia',
     'cluster 29': 'Eastland Gardens, Kenilworth',
     'cluster 30': 'Mayfair, Hillbrook, Mahaning Heights',
     'cluster 31': 'Deanwood, Burrville, Grant Park, Lincoln Heights, Fairmont Heights',
     'cluster 32': 'River Terrace, Benning, Greenway, Fort Dupont',
     'cluster 33': 'Capitol View, Marshall Heights, Benning Heights',
     'cluster 34': 'Twining, Fairlawn, Randle Highlands, Penn Branch, Fort Davis Park, Dupont Park',
     'cluster 35': 'Fairfax Village, Naylor Gardens, Hillcrest, Summit Park',
     'cluster 36': 'Woodland/Fort Stanton, Garfield Heights, Knox Hill',
     'cluster 37': 'Sheridan, Barry Farm, Buena Vista',
     'cluster 38': 'Douglass, Shipley Terrace',
     'cluster 39':'Congress Heights, Bellevue, Washington Highlands'})

   NEIGHBORHOOD  CENSUS_TRACT offensegroup  LONGITUDE  \
8     cluster 9       10500.0     property -77.010660   
20    cluster 8        5800.0      violent -77.022431   
35    cluster 1        4002.0     property -77.041859   
43    cluster 4         100.0     property -77.060134   
45    cluster 1        4001.0      violent -77.043463   

                   END_DATE offense-text    SHIFT     YBLOCK  DISTRICT  WARD  \
8   2015-06-29T21:48:00.000  theft/other  evening  134351.00       1.0   6.0   
20  2015-06-10T13:50:00.000      robbery      day  136884.96       1.0   2.0   
35  2015-12-20T12:05:43.000  theft/other      day  139140.00       3.0   1.0   
43  2015-12-27T15:46:25.000  theft/other  evening  137339.00       2.0   2.0   
45  2015-11-01T03:00:24.000    sex abuse  evening  139315.00       3.0   1.0   

      ...         XBLOCK                                            BLOCK  \
8     ...      399075.00              1100 - 1199 block of half street sw   
20    ...      398054.

In [11]:
#Verify if name was replaced
no_null_crime_df.head()

,NEIGHBORHOOD,CENSUS_TRACT,offensegroup,LONGITUDE,END_DATE,offense-text,SHIFT,YBLOCK,DISTRICT,WARD,...,XBLOCK,BLOCK,START_DATE,CCN,OFFENSE,ANC,REPORT_DAT,METHOD,location,LATITUDE
8,"Southwest Employment Area, Southwest/Waterfron...",10500.0,property,-77.010660,2015-06-29T21:48:00.000,theft/other,evening,134351.00,1.0,6.0,...,399075.00,1100 - 1199 block of half street sw,2015-06-29T21:47:00.000,15098137,theft/other,6D,2015-06-30T02:05:00.000Z,others,"38.876997186422173,-77.010662425191853",38.876989
20,"Downtown, Chinatown, Penn Quarters, Mount Vern...",5800.0,violent,-77.022431,2015-06-10T13:50:00.000,robbery,day,136884.96,1.0,2.0,...,398054.21,700 - 799 block of h street nw,2015-06-10T13:00:00.000,15086313,robbery,2C,2015-06-10T17:50:00.000Z,others,"38.899822390384138,-77.022433674780757",38.899815
35,"Kalorama Heights, Adams Morgan, Lanier Heights",4002.0,property,-77.041859,2015-12-20T12:05:43.000,theft/other,day,139140.00,3.0,1.0,...,396370.00,2300 - 2399 block of 18th street nw,2015-12-19T22:00:27.000,15202729,theft/other,1C,2015-12-20T17:06:13.000Z,others,"38.920131222124674,-77.041861447715945",38.920123
43,"Georgetown, Burleith/Hillandale",100.0,property,-77.060134,2015-12-27T15:46:25.000,theft/other,evening,137339.00,2.0,2.0,...,394784.00,1000 - 1099 block of thomas jefferson street nw,2015-12-27T15:42:52.000,15206326,theft/other,2E,2015-12-27T21:37:01.000Z,others,"38.903899221017866,-77.06013664112929",38.903891
45,"Kalorama Heights, Adams Morgan, Lanier Heights",4001.0,violent,-77.043463,2015-11-01T03:00:24.000,sex abuse,evening,139315.00,3.0,1.0,...,396231.00,1811 - 1852 block of columbia road nw,2015-10-31T22:30:59.000,15206372,sex abuse,1C,2015-12-27T23:46:49.000Z,others,"38.921707093772405,-77.043465279886618",38.921699
